In [2]:
import requests
import json
from dotenv import load_dotenv
import os
load_dotenv()  # have your rpc endpoint in .env file to prevent accidental commit to github

to_address = '0x50f9202e0f1c1577822bd67193960b213cd2f331' # hacker EOA
contract_address = '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881' # hacker contract
url = "https://q.luabase.com/run"

In [3]:
# collect list of erc20 tranfer txs that were sent to the hacker's address
sql = f"select from_address as victim_from_address,token_address as erc20_token_address,to_address as hacker_to_address,transaction_hash, value as amount FROM ethereum.token_transfers AS t WHERE t.to_address = '{to_address}' AND t.block_number >= 15213261"
payload = {
    "block": {
        "details": {
        "sql": sql,
        }
    },
    "api_key": os.environ.get('LUABASE_API_KEY')
}

headers = {"content-type": "application/json"}
response = requests.request("POST", url, json=payload, headers=headers)
transfer_data = response.json()['data']
len(transfer_data)


11

In [4]:
# collect list of txs that have the withdraw erc20 balance method id of victim's addresses & the address called is the hacker's contract. 
sql = f"select from_address,to_address,hash FROM ethereum.transactions AS t where t.input LIKE '%0x9c307de6%' AND t.to_address = '{contract_address}' AND t.block_number >= 15213261"
payload = {
    "block": {
        "details": {
        "sql": sql,
        }
    },
    "api_key": os.environ.get('LUABASE_API_KEY')
}

response = requests.request("POST", url, json=payload, headers=headers)
transaction_data = response.json()['data']
len(transaction_data)

9

In [5]:
# collect victim's addresses
final_data = []
for t in transfer_data:
    for tx in transaction_data:
        if t['transaction_hash'] == tx['hash']:
            # print(tx)
            t['hacker_contract_address'] = tx['to_address']
            final_data.append(t)

print(len(final_data))
final_data[0]

9


{'victim_from_address': '0x1a2a910185467dab98c3c04eee38ac6dbbc3b0d5',
 'erc20_token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
 'hacker_to_address': '0x50f9202e0f1c1577822bd67193960b213cd2f331',
 'transaction_hash': '0xc8e035ac6abd1d7a34a73ed66b7ea15a55d34db762bc073e0a29761975e6c448',
 'amount': 4000917429820000000000,
 'hacker_contract_address': '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881'}

In [20]:
# get erc20 information to calcuate the total assets stolen. 
# TODO: need to filter erc20 duplicates to reduce query time
unique_erc20s = {}

assets = []
for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in unique_erc20s:
        sql = f"select symbol, decimals from ethereum.tokens as t WHERE t.address = '{erc20}'"
        payload = {
            "block": {
                "details": {
                "sql": sql,
                }
            },
            "api_key": os.environ.get('LUABASE_API_KEY')
        }

        # since the response is an array with duplicates, we only need to get first element
        response = requests.request("POST", url, json=payload, headers=headers).json()['data'][0]
        unique_erc20s[erc20] = {'erc20_decimals': response['decimals'], 'erc20_symbol': response['symbol']}
    f.update(unique_erc20s[erc20])

final_data[0]

{'victim_from_address': '0x1a2a910185467dab98c3c04eee38ac6dbbc3b0d5',
 'erc20_token_address': '0x6b175474e89094c44da98b954eedeac495271d0f',
 'hacker_to_address': '0x50f9202e0f1c1577822bd67193960b213cd2f331',
 'transaction_hash': '0xc8e035ac6abd1d7a34a73ed66b7ea15a55d34db762bc073e0a29761975e6c448',
 'amount': 4000917429820000000000,
 'hacker_contract_address': '0x9eb5f8e83359bb5013f3d8eee60bdce5654e8881',
 'erc20_decimals': 18,
 'erc20_name': 'Dai Stablecoin',
 'erc20_symbol': 'DAI'}

In [36]:
# calculate total assets stolen based on the ERC20 information & transaction data retrieved

asset_lost_results = {}

for f in final_data:
    erc20 = f['erc20_token_address']
    if erc20 not in asset_lost_results:
        asset_lost_results[erc20] = {'amount': f['amount'], 'decimals': f['erc20_decimals'], 'symbol': f['erc20_symbol']}
    else:
        asset_lost_results[erc20].update({'amount': f['amount'] + asset_lost_results[erc20]['amount']})

for asset in asset_lost_results.keys():
    asset_lost_results[asset]['amount'] = asset_lost_results[asset]['amount'] / pow(10, asset_lost_results[asset]['decimals'])

asset_lost_results

{'0x6b175474e89094c44da98b954eedeac495271d0f': {'amount': 6458.823763845549,
  'decimals': 18,
  'symbol': 'DAI'},
 '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48': {'amount': 606265.325268,
  'decimals': 6,
  'symbol': 'USDC'}}